In [1]:
import numpy as np
import pandas as pd

In [2]:
larimer_vac_old_file = pd.read_csv('LC-COVID-vaccinations.csv')
larimer_vac_old_file

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,total number of 70+ at least one dose,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,mysteryColumn
0,12/14/2020,16,16,16,16,0,0,NaN,0,NaN,0,0.0,0,0.0,0,0.0,0,NaN,0,0.0
1,12/15/2020,4,20,3,19,1,1,0.0,0,1.0,1,NaN,0,1.0,1,NaN,0,NaN,0,0.0
2,12/16/2020,280,300,279,298,1,2,0.0,0,1.0,2,0.0,0,1.0,2,0.0,0,0.0,0,0.0
3,12/17/2020,939,1239,939,1237,0,2,0.0,0,10.0,12,0.0,0,0.0,2,0.0,0,0.0,0,0.0
4,12/18/2020,990,2229,989,2226,1,3,0.0,0,12.0,24,0.0,0,1.0,3,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,4/7/2021,4359,207056,1323,126509,3036,80547,72.0,29485,69.0,31371,116.0,2896,2593.0,72099,176.0,2550,151.0,3002,NaN
115,4/8/2021,6138,213194,2458,128967,3680,84227,140.0,29625,63.0,31434,142.0,3038,3197.0,75296,168.0,2718,173.0,3175,NaN
116,4/9/2021,5271,218465,1371,130338,3900,88127,82.0,29707,24.0,31458,172.0,3210,3366.0,78662,187.0,2905,175.0,3350,NaN
117,4/10/2021,4065,222530,1843,132181,2222,90349,69.0,29776,21.0,31479,103.0,3313,1880.0,80542,149.0,3054,90.0,3440,NaN


In [3]:
larimer_vac = pd.read_csv('https://speedtest.larimer.org/covid/index.php?file=vaccinations&csv')
larimer_vac

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,mysteryColumn,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,12/14/2020,32,32,24,24,8,8,1.0,1,0.0,...,8,0.0,0,0.0,0,2.0,0.0,8.0,0.0,0.0
1,12/15/2020,16,48,6,30,10,18,0.0,1,1.0,...,17,1.0,1,NaN,0,2.0,0.0,17.0,0.0,0.0
2,12/16/2020,305,353,294,324,11,29,0.0,1,1.0,...,26,1.0,2,0.0,0,13.0,0.0,25.0,1.0,0.0
3,12/17/2020,990,1343,973,1297,17,46,1.0,2,10.0,...,43,0.0,2,0.0,0,44.0,0.0,37.0,1.0,0.0
4,12/18/2020,1050,2393,1040,2337,10,56,2.0,4,14.0,...,53,0.0,2,0.0,0,91.0,0.0,43.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,8/25/2021,532,410465,315,220983,252,206944,11.0,34088,14.0,...,176138,9.0,11453,35.0,7723,NaN,NaN,NaN,NaN,NaN
255,8/26/2021,456,410921,244,221227,239,207183,10.0,34098,12.0,...,176320,22.0,11475,17.0,7740,NaN,NaN,NaN,NaN,NaN
256,8/27/2021,682,411603,336,221563,384,207567,16.0,34114,12.0,...,176601,22.0,11497,47.0,7787,NaN,NaN,NaN,NaN,NaN
257,8/28/2021,415,412018,192,221755,229,207796,8.0,34122,16.0,...,176772,12.0,11509,24.0,7811,NaN,NaN,NaN,NaN,NaN


In [4]:
larimer_vac.head()

,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,mysteryColumn,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,12/14/2020,32,32,24,24,8,8,1.0,1,0.0,...,8,0.0,0,0.0,0,2.0,0.0,8.0,0.0,0.0
1,12/15/2020,16,48,6,30,10,18,0.0,1,1.0,...,17,1.0,1,NaN,0,2.0,0.0,17.0,0.0,0.0
2,12/16/2020,305,353,294,324,11,29,0.0,1,1.0,...,26,1.0,2,0.0,0,13.0,0.0,25.0,1.0,0.0
3,12/17/2020,990,1343,973,1297,17,46,1.0,2,10.0,...,43,0.0,2,0.0,0,44.0,0.0,37.0,1.0,0.0
4,12/18/2020,1050,2393,1040,2337,10,56,2.0,4,14.0,...,53,0.0,2,0.0,0,91.0,0.0,43.0,1.0,1.0


In [5]:
larimer_vac.shape

(259, 24)

In [6]:
len(larimer_vac)

259

In [7]:
larimer_vac.count()

Date                                                          259
daily number of doses received by Larimer County residents    259
total number of doses recevied by residents                   259
daily number of residents receiving first dose                259
total number of residents receiving first dose                259
daily number of residents vaccinated                          259
total number of residents vaccinated                          259
daily number of 70+ vaccinated                                257
total number of 70+ vaccinated                                259
daily number of 70+ at least one dose                         257
total number of 70+ at least one dose                         259
daily number of Latinx residents vaccinated                   258
total of Latinx residents vaccinated                          259
daily number of White non-Latinx residents vaccinated         258
total of White non-Latinx residents vaccinated                259
daily numb

In [8]:
larimer_vac.describe()

,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,total number of 70+ at least one dose,...,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,mysteryColumn,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
count,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,257.000000,259.000000,257.000000,259.000000,...,259.000000,257.000000,259.000000,255.000000,259.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,1591.397683,234911.953668,856.532819,133125.629344,802.559846,110463.536680,132.778210,23758.084942,137.813230,27729.714286,...,95753.872587,44.809339,5587.969112,30.654902,3560.698842,4193.379121,2856.582418,60951.148352,2916.434066,3083.796703
std,1463.573555,153011.913173,882.308895,80045.004174,869.858983,80737.981706,271.834511,13387.890190,263.857302,11499.726020,...,68885.526359,54.109974,4570.270755,31.180174,2926.440314,3410.610353,2836.083754,55015.660368,3117.484599,3125.201338
min,1.000000,32.000000,1.000000,24.000000,0.000000,8.000000,0.000000,1.000000,0.000000,0.000000,...,8.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,8.000000,0.000000,0.000000
25%,432.000000,77107.500000,242.000000,53693.500000,178.500000,23419.000000,6.000000,10927.000000,8.000000,27296.500000,...,21066.500000,12.000000,837.000000,12.000000,608.000000,983.500000,347.250000,8216.750000,319.500000,315.750000
50%,1010.000000,280447.000000,513.000000,172462.000000,436.000000,120323.000000,16.000000,31908.000000,19.000000,33896.000000,...,106872.000000,23.000000,5121.000000,21.000000,3104.000000,3107.000000,1766.500000,46353.500000,1464.000000,1849.500000
75%,2674.500000,386036.500000,1148.000000,207171.000000,1145.000000,194770.000000,91.000000,33667.000000,117.000000,34866.000000,...,167129.000000,51.000000,10595.000000,38.500000,6693.000000,7779.250000,5341.500000,114031.500000,5436.500000,5527.000000
max,6671.000000,412172.000000,4492.000000,221842.000000,4167.000000,207863.000000,1752.000000,34124.000000,1596.000000,35418.000000,...,176825.000000,242.000000,11516.000000,159.000000,7817.000000,9728.000000,8346.000000,154480.000000,9046.000000,9200.000000


In [9]:
# make a backup file
larimer_vac.to_csv('larimer_vac_backup.csv')